In [7]:
import imdb as db
from sqlalchemy import select
from sqlalchemy.orm import Session

In [8]:
stm = (
    select(db.Actor)
    .join(db.Actor.roles)
    .join(db.Role.movie)
    .join(db.Movie.genres)
    .where(db.MoviesGenres.genre == 'War')
)

In [40]:
s = Session(db.engine)
res = s.execute(stm)
for i in res.scalars():
  print(i.last_name)



2022-10-01 17:57:40,165 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-01 17:57:40,166 INFO sqlalchemy.engine.Engine SELECT actors.id, actors.first_name, actors.last_name, actors.gender 
FROM actors JOIN roles ON actors.id = roles.actor_id JOIN movies ON movies.id = roles.movie_id JOIN movies_genres ON movies.id = movies_genres.movie_id 
WHERE movies_genres.genre = ?
2022-10-01 17:57:40,167 INFO sqlalchemy.engine.Engine [cached since 710.7s ago] ('War',)
Achara
Adams
Aiken
Albridge
Alden
Alexander
Alexander
Amada
Anderson
Anderson
Anderson
Appiano
Arledge
Armstrong
Assböck
Assmann
Ates
Avrahami
Ayres
Bacon
Bagdasarian
Bailey
Baker
Baker
Bakewell
Bakewell
Baldwin
Baldwin
Bannen
Bardette
Barlotti
Barnett
Barnett
Barracks
Barredo
Bau
Beaton
Beck
Beckham
Beddows
Beeby
Beedle
Bell
Bemme
Bender
Benedict
Benn
Bennett
Bennett
Berenger
Bergmann
Berkoff
Berry
Billington
Bindon
Bista
Boa
Bolt
Bond
Boos
Booth
Bottoms
Boucher
Boyle
Bradecki
Brand
Brando
Breese
Brejdygant
Briggs
Brooks
Brook